## Web Crawler de criticas de filmes

In [1]:
import pandas as pd
import numpy as np
import requests as rq
from bs4 import BeautifulSoup
import urllib.robotparser as robotparser


### Paginas

In [2]:
paginas = [('Jornal_da_Paraiba','www.jornaldaparaiba.com.br/'),('Cineclick','www.cineclick.com.br/'), 
('Cinema_com_Rapadura','cinemacomrapadura.com.br/'),
('Plano_Critico','www.planocritico.com/'),('Plano_Aberto','www.planoaberto.com.br/'),
('Cinemasim','www.cinemasim.com.br/'), ('Pocilga','pocilga.com.br/')]

#('Omelete','http://www.omelete.com.br/'),('Elpais','https://brasil.elpais.com/', ('Cinema_em_Cena','http://cinemaemcena.cartacapital.com.br/')),

### Pegando Robots.txt

   ##### Site http://cinemaemcena.cartacapital.com.br/ não possui o arquivo

### Coletando paginas

In [ ]:
from urllib.request import urlopen
from bs4.dammit import EncodingDetector
from collections import deque
import re
import time
import random
from random import randrange


def get_links(page):
    home_page =  page[1]
    print(home_page)
    ##Carrega o robots.txt
    rp = robotparser.RobotFileParser()
    rp.set_url("http://" +home_page +"robots.txt")
    rp.read()
    
    fila_paginas = deque([home_page])
    lista_paginas = [home_page]
    
    headers = {
        'Accept-Encoding': 'gzip, deflate, sdch',
        'Accept-Language': 'pt-BR',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Cache-Control': 'max-age=0',
        'Connection': 'keep-alive',
    }

    cont = 0
    while fila_paginas:
        page_link = fila_paginas.popleft()
        #print(page_link)
        cont = cont+1 
        #print("asdajsd")
        resp = rq.get("http://"+page_link,allow_redirects=False, headers=headers)
        http_encoding = resp.encoding if 'charset' in resp.headers.get('content-type', '').lower() else None
        html_encoding = EncodingDetector.find_declared_encoding(resp.content, is_html=True)
        encoding = html_encoding or http_encoding
        soup = BeautifulSoup(resp.content, from_encoding=encoding)
        links = []
        if not (resp.content):
            print("oi")
            continue
        pattern = re.compile("^(/|http)")
        for link in soup.find_all("a", href=pattern):
            href = link.get('href')           
            if href.endswith(".xml") or href.endswith(".jpg") or href.endswith(".pnj"):
                continue
            if href.startswith("//"):
                href = href[2:] 
            if href.startswith("/"):
                href = home_page + href[1:]                
            href = href.replace("https://", "")
            href = href.replace("http://", "")
            if not (href.startswith(home_page)):
                continue                               
            if href not in lista_paginas:                    
                if len(lista_paginas) < 1000:
                    if rp.can_fetch("*", href) or (page[0] == 'Cinema_em_Cena'):
                        lista_paginas.append(href)
                        fila_paginas.append(href)
        page_link = page_link.replace("https://", "")
        page_link = page_link.replace("http://", "")
        page_link = page_link.replace("/", "_")                        
        with open("pages/bfs/"+page[0]+"/"+page_link+".html", 'wb') as f:
            f.write(resp.content)
        if (cont % 10) == 0:
            time.sleep(randrange(3)+2)
        if (cont == 500):
            time.sleep(60)
    #print(lista_paginas)


for p in paginas:
    get_links(p)

www.jornaldaparaiba.com.br/


/home/leonardo/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/leonardo/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
www.cineclick.com.br/
oi
cinemacomrapadura.com.br/
www.planocritico.com/
oi
